In [6]:
import geopandas as gpd
import pandas as pd
import panel as pn 
import holoviews as hv

Census reorienting

In [2]:
# read in census.geojson into pandas dataframe from data folder that is in a different directory
census = gpd.read_file('../data/census2.geojson')
census.head()

,NAME,PLACEFIPS,year,entityID,pl_age_total_und18,pl_age_total_18_64,pl_age_total_ov65,pl_pop_total,pl_age_white_und18,pl_age_white_18_64,...,st_pov_asian_18,st_pov_pacisl_tot,st_pov_pacisl_18,st_pov_other_tot,st_pov_other_18,st_pov_mult_tot,st_pov_mult_18,st_pov_his_tot,st_pov_his_18,geometry
0,Santee,3143475,2011-06-01,43475,44.9,54.5,0.6,334,0.0,94.4,...,3.6,50.8,32.0,23.3,11.5,23.7,14.3,24.3,12.7,"POLYGON ((-97.85201 42.84192, -97.85185 42.841..."
1,Rosalie,3142250,2011-06-01,42250,24.7,53.7,21.6,231,13.8,59.6,...,3.6,50.8,32.0,23.3,11.5,23.7,14.3,24.3,12.7,"POLYGON ((-96.50927 42.06107, -96.50932 42.060..."
2,Macy,3130170,2011-06-01,30170,53.5,42.4,4.1,851,0.0,15.4,...,3.6,50.8,32.0,23.3,11.5,23.7,14.3,24.3,12.7,"POLYGON ((-96.37321 42.12605, -96.37278 42.125..."
3,Walthill,3151245,2011-06-01,51245,33.7,54.9,11.4,861,10.1,61.2,...,3.6,50.8,32.0,23.3,11.5,23.7,14.3,24.3,12.7,"POLYGON ((-96.49161 42.15520, -96.49145 42.151..."
4,Winnebago,3153275,2011-06-01,53275,48.0,47.0,5.0,996,0.0,75.0,...,3.6,50.8,32.0,23.3,11.5,23.7,14.3,24.3,12.7,"POLYGON ((-96.46833 42.24599, -96.46834 42.245..."


List of ethnicities

In [3]:
ethnic_list = ["white", "black", "nat", "asian", "pacisl", "other", "mult", "his", "total"]

Loop through list of ethnicities, subset dataframe, and add to list of dataframes

In [4]:
# create list of dataframes to concatenate
df_list = []

for ethnicity in ethnic_list:

    # replace column name
    col_repl = "_" + ethnicity

    # Get the columns that have "bl" in them
    sub_cols = [col for col in census.columns.tolist() if col_repl in col ]
    sub_cols = sub_cols + ["NAME", "year", "entityID", "geometry"]

    # subset the dataframe to only include the columns that have "bl" in them
    census_sub = census[sub_cols]
    census_sub.columns = census_sub.columns.str.replace(col_repl, "")

    # add ethnicity column and populate it with ethnicity
    census_sub = census_sub.assign(ethnicity = ethnicity)

    # append to list
    df_list.append(census_sub)

In [5]:
# concat geodataframes into census_oriented
census_oriented = pd.concat(df_list)
census_oriented.head()

,pl_age_und18,pl_age_18_64,pl_age_ov65,pl_pop,pl_medage,pl_pov_tot,pl_pov_18,us_age_und18,us_age_18_64,us_age_ov65,...,st_age_ov65,st_pop,st_medage,st_pov_tot,st_pov_18,NAME,year,entityID,geometry,ethnicity
0,0.0,94.4,5.6,18,62.8,0.0,0.0,22.2,64.9,14.8,...,14.6,1600661,38.2,10.1,3.0,Santee,2011-06-01,43475,"POLYGON ((-97.85201 42.84192, -97.85185 42.841...",white
1,13.8,59.6,26.6,188,45.3,17.6,0.0,22.2,64.9,14.8,...,14.6,1600661,38.2,10.1,3.0,Rosalie,2011-06-01,42250,"POLYGON ((-96.50927 42.06107, -96.50932 42.060...",white
2,0.0,15.4,84.6,26,84.1,28.6,0.0,22.2,64.9,14.8,...,14.6,1600661,38.2,10.1,3.0,Macy,2011-06-01,30170,"POLYGON ((-96.37321 42.12605, -96.37278 42.125...",white
3,10.1,61.2,28.7,129,61.1,6.2,0.0,22.2,64.9,14.8,...,14.6,1600661,38.2,10.1,3.0,Walthill,2011-06-01,51245,"POLYGON ((-96.49161 42.15520, -96.49145 42.151...",white
4,0.0,75.0,25.0,20,33.5,33.3,0.0,22.2,64.9,14.8,...,14.6,1600661,38.2,10.1,3.0,Winnebago,2011-06-01,53275,"POLYGON ((-96.46833 42.24599, -96.46834 42.245...",white


Build charts for panel app

In [ ]:
# define function to create holoviz bar chart
def age_group_chart():

    